In [48]:
import pandas as pd
import numpy as np
import string
import scipy

import re
from pprint import pprint

import nltk
from nltk.stem import PorterStemmer

import gensim, logging
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, ldamodel

import spacy
# For future ref, the only install that's ever worked for spacy: conda install -c conda-forge spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [32]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize

In [6]:
abbott = open('abbott.txt','r')
#print(abbott.read())

abbott = [re.sub("\'", "", sent) for sent in abbott]
pprint(abbott[:1])

#abbott_string = abbott.read()
#atokens = nltk.word_tokenize(abbott_string)
    
#print(atokens)

['Berenice Abbott (July 17, 1898 – December 9, 1991),[2] née Bernice Alice '
 'Abbott, was an American photographer best known for her portraits of '
 'between-the-wars 20th-century cultural figures, New York City photographs of '
 'architecture and urban design of the 1930s, and science interpretation in '
 'the 1940s–1960s.Abbott was born in Springfield, Ohio,[3] the youngest of '
 'four children.[4][unreliable source] and brought up there by her divorced '
 'mother, née Lillian Alice Bunn (m. Charles E. Abbott in Chillicothe OH, '
 '1886). She took college preparatory classes and graduated in 1917.[4] She '
 'attended Ohio State University for two semesters, but left in early 1918 '
 'because her German professor was dismissed because he was a German teaching '
 'an English class.[5] Abbot wanted to take a job in journalism, but later '
 'lost interest in it and followed the path of photography because of the '
 'interaction with Eugene O’Neill after that she became attached to '
 '

In [33]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
 
data_words = list(sent_to_words(abbott))

print(data_words[:1])

[['berenice', 'abbott', 'july', 'december', 'nee', 'bernice', 'alice', 'abbott', 'was', 'an', 'american', 'photographer', 'best', 'known', 'for', 'her', 'portraits', 'of', 'between', 'the', 'wars', 'th', 'century', 'cultural', 'figures', 'new', 'york', 'city', 'photographs', 'of', 'architecture', 'and', 'urban', 'design', 'of', 'the', 'and', 'science', 'interpretation', 'in', 'the', 'abbott', 'was', 'born', 'in', 'springfield', 'ohio', 'the', 'youngest', 'of', 'four', 'children', 'unreliable', 'source', 'and', 'brought', 'up', 'there', 'by', 'her', 'divorced', 'mother', 'nee', 'lillian', 'alice', 'bunn', 'charles', 'abbott', 'in', 'chillicothe', 'oh', 'she', 'took', 'college', 'preparatory', 'classes', 'and', 'graduated', 'in', 'she', 'attended', 'ohio', 'state', 'university', 'for', 'two', 'semesters', 'but', 'left', 'in', 'early', 'because', 'her', 'german', 'professor', 'was', 'dismissed', 'because', 'he', 'was', 'german', 'teaching', 'an', 'english', 'class', 'abbot', 'wanted', 'to

In [9]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)


/Users/marieskoczylas/anaconda3/envs/nlp_environ/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [34]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['berenice', 'abbott', 'july', 'december', 'nee', 'bernice', 'alice', 'abbott', 'was', 'an', 'american', 'photographer', 'best', 'known', 'for', 'her', 'portraits', 'of', 'between', 'the', 'wars', 'th', 'century', 'cultural', 'figures', 'new', 'york', 'city', 'photographs', 'of', 'architecture', 'and', 'urban', 'design', 'of', 'the', 'and', 'science', 'interpretation', 'in', 'the', 'abbott', 'was', 'born', 'in', 'springfield', 'ohio', 'the', 'youngest', 'of', 'four', 'children', 'unreliable', 'source', 'and', 'brought', 'up', 'there', 'by', 'her', 'divorced', 'mother', 'nee', 'lillian', 'alice', 'bunn', 'charles', 'abbott', 'in', 'chillicothe', 'oh', 'she', 'took', 'college', 'preparatory', 'classes', 'and', 'graduated', 'in', 'she', 'attended', 'ohio', 'state', 'university', 'for', 'two', 'semesters', 'but', 'left', 'in', 'early', 'because', 'her', 'german', 'professor', 'was', 'dismissed', 'because', 'he', 'was', 'german', 'teaching', 'an', 'english', 'class', 'abbot', 'wanted', 'to'

In [35]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatizations(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://space.io/api/annotation"""
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [36]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

#nlp = spacy.load('en', disable=['parser', 'ner']) # Doesn't work because spacy and 'en' don't work

#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

[['berenice', 'abbott', 'july', 'december', 'nee', 'bernice', 'alice', 'abbott', 'american', 'photographer', 'best', 'known', 'portraits', 'wars', 'th', 'century', 'cultural', 'figures', 'new', 'york', 'city', 'photographs', 'architecture', 'urban', 'design', 'science', 'interpretation', 'abbott', 'born', 'springfield', 'ohio', 'youngest', 'four', 'children', 'unreliable', 'source', 'brought', 'divorced', 'mother', 'nee', 'lillian', 'alice', 'bunn', 'charles', 'abbott', 'chillicothe', 'oh', 'took', 'college', 'preparatory', 'classes', 'graduated', 'attended', 'ohio', 'state', 'university', 'two', 'semesters', 'left', 'early', 'german', 'professor', 'dismissed', 'german', 'teaching', 'english', 'class', 'abbot', 'wanted', 'take', 'job', 'journalism', 'later', 'lost', 'interest', 'followed', 'path', 'photography', 'interaction', 'eugene', 'neill', 'became', 'attached', 'photography', 'abbott', 'also', 'well', 'known', 'photographer', 'paris', 'migrants', 'became', 'assistant', 'man', 'ra

In [37]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 10), (2, 4), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 2), (18, 1), (19, 1), (20, 2), (21, 2), (22, 6), (23, 4), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 4), (33, 1), (34, 3), (35, 3), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 4), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 4), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 3), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 4), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 2), (103, 2), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

In [38]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abbot', 1),
  ('abbott', 10),
  ('abbotts', 4),
  ('able', 1),
  ('academie', 1),
  ('academy', 1),
  ('according', 1),
  ('acquired', 2),
  ('addition', 2),
  ('adopted', 1),
  ('alice', 2),
  ('allowed', 1),
  ('also', 1),
  ('american', 1),
  ('andre', 2),
  ('anything', 1),
  ('architecture', 2),
  ('archive', 2),
  ('art', 1),
  ('artistic', 1),
  ('arts', 2),
  ('assistant', 2),
  ('atget', 6),
  ('atgets', 4),
  ('attached', 1),
  ('attended', 1),
  ('au', 1),
  ('bac', 1),
  ('barnes', 1),
  ('battery', 1),
  ('beach', 1),
  ('beautiful', 1),
  ('became', 4),
  ('behalf', 1),
  ('berenice', 3),
  ('berlin', 3),
  ('bernice', 1),
  ('best', 1),
  ('book', 2),
  ('born', 1),
  ('bowery', 1),
  ('brought', 1),
  ('brussels', 1),
  ('bunn', 1),
  ('buy', 1),
  ('calmettes', 1),
  ('century', 1),
  ('champs', 1),
  ('changing', 2),
  ('charles', 1),
  ('chaumiere', 1),
  ('children', 1),
  ('chillicothe', 1),
  ('city', 4),
  ('class', 1),
  ('classes', 1),
  ('cocteau', 1),
  (

In [39]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [40]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.002*"new" + 0.002*"abbott" + 0.002*"photography" + 0.002*"work" + '
  '0.002*"york" + 0.002*"abbotts" + 0.002*"city" + 0.002*"photographs" + '
  '0.002*"paris" + 0.002*"studio"'),
 (1,
  '0.002*"work" + 0.002*"abbott" + 0.002*"man" + 0.002*"ray" + 0.002*"became" '
  '+ 0.002*"atget" + 0.002*"paris" + 0.002*"photography" + 0.002*"photographs" '
  '+ 0.002*"berlin"'),
 (2,
  '0.002*"york" + 0.002*"new" + 0.002*"work" + 0.002*"abbott" + '
  '0.002*"photographs" + 0.002*"city" + 0.002*"photography" + 0.002*"abbotts" '
  '+ 0.002*"paris" + 0.001*"studio"'),
 (3,
  '0.023*"abbott" + 0.021*"new" + 0.019*"york" + 0.019*"work" + '
  '0.016*"photographs" + 0.015*"city" + 0.015*"photography" + 0.012*"abbotts" '
  '+ 0.006*"project" + 0.006*"paris"'),
 (4,
  '0.002*"abbott" + 0.002*"new" + 0.002*"york" + 0.002*"work" + '
  '0.002*"photography" + 0.002*"photographs" + 0.002*"city" + 0.002*"abbotts" '
  '+ 0.002*"project" + 0.002*"paris"'),
 (5,
  '0.002*"abbott" + 0.002*"york" + 0.002*"ci

In [41]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# A measure of how good the model is. The lower the better. Mine here is -7.53

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.525054255166115

Coherence Score:  0.3062669383365394


In [53]:
pyLDAvis.enable_notebook() 
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word) 
vis


/Users/marieskoczylas/anaconda3/envs/nlp_environ/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
3      0.063317 -1.633755e-08       1        1  99.921379
1     -0.000822  1.771718e-04       2        1   0.004778
4     -0.002040 -1.013579e-05       3        1   0.004152
13    -0.000945 -9.576870e-05       4        1   0.004152
12    -0.001175 -7.792748e-05       5        1   0.004151
9     -0.002767 -3.170708e-06       6        1   0.004148
0     -0.002931  1.880163e-05       7        1   0.004148
2     -0.003139  7.654064e-06       8        1   0.004147
5     -0.003340  1.557511e-05       9        1   0.004146
11    -0.003436  4.138522e-06      10        1   0.004144
7     -0.003991  2.242620e-05      11        1   0.004141
19    -0.003839  4.676401e-05      12        1   0.004139
8     -0.004120  1.580093e-05      13        1   0.004139
15    -0.004242  1.966275e-05      14        1   0.004138
18    -0.003162 -4.602798e-05      15        1   0.004138
17    -0.004005 -1.521139e-05      16        1   0.004134
10    -0.005249 -4.755671e-06      17        1   0.004116
16    -0.005725  9.610067e-06      18        1   0.004089
14    -0.003422 -6.373048e-05      19        1   0.003818
6     -0.004966 -2.086060e-05      20        1   0.003798, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
1     Default  28.000000        abbott  28.000000  30.0000  30.0000
169   Default  26.000000           new  26.000000  29.0000  29.0000
274   Default  23.000000          work  23.000000  28.0000  28.0000
282   Default  23.000000          york  23.000000  27.0000  27.0000
189   Default  19.000000   photographs  19.000000  26.0000  26.0000
53    Default  18.000000          city  18.000000  25.0000  25.0000
190   Default  18.000000   photography  18.000000  24.0000  24.0000
2     Default  14.000000       abbotts  14.000000  23.0000  23.0000
22    Default   7.000000         atget   7.000000  22.0000  22.0000
176   Default   7.000000         paris   7.000000  21.0000  21.0000
587   Default   7.000000       project   7.000000  20.0000  20.0000
240   Default   6.000000        studio   6.000000  19.0000  19.0000
38    Default   6.000000          book   6.000000  18.0000  18.0000
208   Default   5.000000           ray   5.000000  17.0000  17.0000
515   Default   5.000000          made   5.000000  16.0000  16.0000
254   Default   5.000000          time   5.000000  15.0000  15.0000
161   Default   5.000000        museum   5.000000  14.0000  14.0000
259   Default   5.000000           two   5.000000  13.0000  13.0000
516   Default   5.000000         maine   5.000000  12.0000  12.0000
140   Default   5.000000         later   5.000000  11.0000  11.0000
525   Default   4.000000    mccausland   4.000000  10.0000  10.0000
416   Default   4.000000           era   4.000000   9.0000   9.0000
335   Default   4.000000        camera   4.000000   8.0000   8.0000
619   Default   4.000000    scientific   4.000000   7.0000   7.0000
150   Default   4.000000           man   4.000000   6.0000   6.0000
165   Default   4.000000     negatives   4.000000   5.0000   5.0000
16    Default   4.000000  architecture   4.000000   4.0000   4.0000
48    Default   4.000000      changing   4.000000   3.0000   3.0000
32    Default   4.000000        became   4.000000   2.0000   2.0000
194   Default   4.000000      portrait   4.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
282   Topic20   0.000072          york  23.371981  -2.5081  -6.4859
189   Topic20   0.000071   photographs  19.483652  -2.3460  -6.5057
1     Topic20   0.000071        abbott  28.226973  -2.7130  -6.5020
169   Topic20   0.000071           new  26.296827  -2.6502  -6.5100
274   Topic20   0.000070          work  23.364918  -2.5378  -6.5158
53    Topic20   0.000070          city  18.521038  -2.3093  -6.5197
2     Topic20   0.000069     

In [ ]:
# Each bubble represents a topic. The larger the bubble, the more prevalent the topic. 
#A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart
# instead of being clustered in one region. The next chart: salient keywords that form the selected topic. 
# Next, I could use Mallet's verion of LDA algorithm to improve the model and then focus on optimal # of topics.